# Agente Auto


This notebook presents an agent-based model that simulates the propagation of a disease through a network.
It demonstrates how to use the [agentpy](https://agentpy.readthedocs.io) package to create and visualize networks, use the interactive module, and perform different types of sensitivity analysis. 

In [1]:
# Model design
import agentpy as ap
import networkx as nx 
import random 

# Visualization
import matplotlib.pyplot as plt 
import seaborn as sns
import IPython

## About the model 
Es



## Defining the model

In [8]:
class Car(ap.Agent):
    
    def setup(self):  
        """ Initialize a new variable at agent creation. """
        self.condition = 0  # Conduciendo = 0, Standby = 1
        self.turn_chance = self.model.random
        self.grid = self.model.grid
        self.dir = 0 #Horizontal = 0, Vertical = 1
        
        
        
    def drive(self):
        """ Manejar """
        if (self.grid.positions[self] == (0, int(self.p.gridsize/2)) or 
           self.grid.positions[self] == (0, int(self.p.gridsize/2+1))):
           self.dir = 1
           
        if self.condition == 0 and self.dir == 0:  #Horizontal
            self.grid.move_by(self, (0,1))
        if self.condition == 0 and self.dir == 1: #Vertical
            self.grid.move_by(self, (1,0))
        
        rng = self.model.random
        if ((self.grid.positions[self] == (self.p.gridsize/2+1,self.p.gridsize/2) and self.dir == 0) or
            (self.grid.positions[self] == (self.p.gridsize/2,self.p.gridsize/2+1) and self.dir == 1) and  # street is Turnable
            self.p.turn_chance > rng.random()):
            if self.dir == 0:
                self.dir = 1
            else:
                self.dir = 0
                

        

Traffic Light

In [9]:
class TLight(ap.Agent):
    
    def setup(self):
        self.condition = 1          # Rojo = 1, Verde = 2
        self.grid = self.model.grid

    #def pos(self):
        #self.pos = self.grid.pos[self]

    

In [4]:
class CarModel(ap.Model):
    
    def setup(self):

        gridsize = self.p['gridsize']

        rng = self.model.random
        """ Initialize the agents and network of the model. """
        self.grid = ap.Grid(self, (self.p.gridsize, self.p.gridsize), torus= True, track_empty= True)

        # self.agents = Car
        self.agents = ap.AgentList(self, 1, Car)
        if 0.5 > rng.random():
            self.grid.add_agents(self.agents, [(int(self.p.gridsize/2),0)])
        else:
            self.grid.add_agents(self.agents, [(int(self.p.gridsize/2+1),0)])

        # self.semaforo = TLight
        self.semaforo = ap.AgentList(self, 4, TLight)
        self.grid.add_agents(self.semaforo, positions = [
            (int((gridsize/2)-2), int((gridsize/2)+2)),
            (int((gridsize/2)+2), int((gridsize/2)-2)),
            (int((gridsize/2)-2), int((gridsize/2)-2)),
            (int((gridsize/2)+2), int((gridsize/2)+2))
        ])
        
        
    def step(self):   
        """ Define the models' events per simulation step. """
        self.agents.drive()
        rng = self.model.random
        if self.p.car_chance > rng.random() and len(self.agents) < self.p.cars: 
            self.new_agents = ap.AgentList(self, 1, Car)
            self.agents.extend(self.new_agents)
            if 0.5 > rng.random():
                if 0.5 > rng.random():
                    self.grid.add_agents(self.new_agents, [(int(self.p.gridsize/2),0)], empty=True)
                else:
                    self.grid.add_agents(self.new_agents, [(int(self.p.gridsize/2+1),0)], empty=True)
            else:
                if 0.5 > rng.random():
                    self.grid.add_agents(self.new_agents, [(0, int(self.p.gridsize/2))], empty=True)
                    
                else:
                    self.grid.add_agents(self.new_agents, [(0, int(self.p.gridsize/2+1))], empty=True)
                    
                
        

    
        

## Running a simulation

In [10]:
parameters = {  
    'gridsize': 24,
    'cars': 9,
    'steps': 100,
    'turn_chance': 0.2,
    'car_chance': 0.1,
}

model = CarModel(parameters)
results = model.run() 

Completed: 100 steps
Run time: 0:00:00.020075
Simulation finished


## Analyzing results

In [11]:
results

DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 5 keys
'reporters': DataFrame with 1 variable and 1 row
}

In [12]:
def animation_plot(model, ax):
    attr_grid = model.grid.attr_grid('condition')
    # 0: carro
    # 2: semáforo rojo
    # 3: semáforo verde
    color_dict = {0: '#000000', 1: '#D62C2C', 2: '#21D41E', None: '#EAE5E2'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Car model \n Time-step: {model.t}")

fig, ax = plt.subplots()
model = CarModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())